# **Task: Using the Drone's front camera, detect an AR tag, identify the Tag ID, then compute the drone's pose from the AR tag's information**

Steps:

1. Use a program to extract the AR tag and convert to a matrix to find the TAG ID
2. Corrospond the AR tag to known location values in a pre-determined index
3. Using the points of the AR tag, output the drone's pose in a 6 dimensional vector

# Imports

In [1]:
from __future__ import print_function
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import cv2
import cv2.aruco as aruco
import os
import glob
from matplotlib.patches import Rectangle
from scipy.stats import linregress
import matplotlib.patches as patches
from matplotlib.lines import Line2D

# Step 1

**Use a program to extract the AR tag to find the tag ID**

In [ ]:
folderpath = "ARTags/"
imagepaths = glob.glob(os.path.join(folderpath, "*.png"))
kernel = np.ones((3, 3),np.uint8)
print(cv2.__version__)

for path in imagepaths:
    image = cv2.imread(path)

    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    image = cv2.convertScaleAbs(gray, alpha=1.0, beta=-50)
    image = cv2.erode(image, kernel)


    aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_5X5_100)

    detectorParams = aruco.DetectorParameters()
    detectorParams.aprilTagCriticalRad = 0.1 #default 0.17
    detectorParams.aprilTagMaxLineFitMse = 100 #default 10
    detectorParams.aprilTagMaxNmaxima = 15 #default 10
    detectorParams.polygonalApproxAccuracyRate = 0.01 #default 0.03
    detectorParams.useAruco3Detection = True
    detector = aruco.ArucoDetector(aruco_dict, detectorParams)
    corners, ids, rejected = detector.detectMarkers(gray)

    if ids is not None: 

        print(f"Detected ID: {ids.flatten()} on image: {path}")
        image = aruco.drawDetectedMarkers(image, corners, ids)
    else:
        print(f"No markers found for image: {path}")
        # fig, ax = plt.subplots()
        # ax.imshow(image, cmap='gray')

        plt.show()





4.12.0
<class 'numpy.ndarray'>
Detected ID: [87] on image: ARTags/id87.png
<class 'numpy.ndarray'>
Detected ID: [93] on image: ARTags/image5.png
<class 'numpy.ndarray'>
Detected ID: [95] on image: ARTags/id95.png
<class 'numpy.ndarray'>
Detected ID: [95] on image: ARTags/id95_2.png
<class 'numpy.ndarray'>
Detected ID: [95] on image: ARTags/image3.png
<class 'NoneType'>
No markers found for image: ARTags/image.png
<class 'numpy.ndarray'>
Detected ID: [93] on image: ARTags/image6.png
<class 'numpy.ndarray'>
Detected ID: [87] on image: ARTags/id87_2.png
<class 'numpy.ndarray'>
Detected ID: [93] on image: ARTags/id93_1.png
<class 'NoneType'>
No markers found for image: ARTags/id87_1.png


KeyboardInterrupt: 

# Step 2 
**Corrospond the AR tag to a predetermined location**

# Step 3

**Find the drone's position relative to the tag's position**

In [ ]:
MARKER_LENGTH = 0.2667 #Meters
CAMERA_MATRIX = np.array([])
DIST_COEFFS = []

def rotationMatrixToEulerAngles(R):

    sy = np.linalg.norm([R[0, 0], R[1, 0]])
    singular = sy < 1e-6

    if not singular:
        x = np.arctan2(R[2, 1], R[2, 2])  
        y = np.arctan2(-R[2, 0], sy)      
        z = np.arctan2(R[1, 0], R[0, 0])  
    else:
        x = np.arctan2(-R[1, 2], R[1, 1])
        y = np.arctan2(-R[2, 0], sy)
        z = 0

    return np.array([x, y, z])


def func(corners):

    rvecs, tvecs, _ = aruco.estimatePoseSingleMarkers(corners, MARKER_LENGTH, CAMERA_MATRIX, DIST_COEFFS)

    R_marker_to_cam, _ = cv2.Rodrigues(rvecs)

    R_cam_to_marker = R_marker_to_cam.T

    t_cam_to_marker = -R_cam_to_marker @ tvecs #The position of the camera in the marker's coordinate frame

    rvec_cam_to_marker, _ = cv2.Rodrigues(R_cam_to_marker) #The rotation of the camera in relation to the marker (in marker frame)

    x, y, z = rotationMatrixToEulerAngles(R_cam_to_marker) #Roll, pitch, yaw
        


# Step 4

**Write the ros node**

In [ ]:

import rclpy
from rclpy.node import Node
import cv2
from std_msgs.msg import String
from std_msgs.msg import Int32
from sensor_msgs.msg import Image
from cv_bridge import CvBridge, CvBridgeError
import cv2.aruco as aruco
import sys

CAR_TAG_ID = 96
CAR_MARKER_LENGTH = 0 #TBD -- Meters
MARKER_LENGTH = 0.2667 #Meters
CAMERA_MATRIX = np.array([])
DIST_COEFFS = ()
KERNEL = np.ones((5, 5),np.uint8)

class ARTagDetector(Node):
    def __init__(self):
        super().__init__('artag_detector')
        self.get_logger().info('ARTag Detector has been initialized')
        
        #Create camera subscription, 50 milli second wait
        self.camera_sub = self.create_subscription(
            Image,
            '/world/line_following_track/model/x500_mono_cam_down_0/link/camera_link/sensor/imager/image',
            self.camera_sub_cb,
            50
        )

        self.tagid_pub = self.create_publisher(String, '/', 1)
        self.position_pub = self.create_publisher(Int32, '/line/detector_image', 1)
        self.bridge = CvBridge()

    """
    camera callback
    Publishes to tag id and position pub
    
    """
    def camera_sub_cb(self, msg):
        # Convert Image msg to OpenCV image
        image = self.bridge.imgmsg_to_cv2(msg, "mono8")

        # Detect line in the image. detect returns a parameterize the line (if one exists)
        tagids, corners = self.detect_id(image)
        if tagids is None:
            self.get_logger().info('No ARTag found!')
        elif np.isin(tagids, CAR_TAG_ID):
            self.get_logger().info(f"Car ARTag Detected! Awaiting response from car...")
            #To be implemented
        else:
            self.get_logger().info(f'ARTag Detected! Detected: {tagids.flatten()}')

        #Rotate to Euler angles
        def rotationMatrixToEulerAngles(R):
            sy = np.linalg.norm([R[0, 0], R[1, 0]])
            singular = sy < 1e-6

            if not singular:
                x = np.arctan2(R[2, 1], R[2, 2])  
                y = np.arctan2(-R[2, 0], sy)      
                z = np.arctan2(R[1, 0], R[0, 0])  
            else:
                x = np.arctan2(-R[1, 2], R[1, 1])
                y = np.arctan2(-R[2, 0], sy)
                z = 0
            return np.array([x, y, z])

        # Aruco Estimate pose
        rvecs, tvecs, _ = aruco.estimatePoseSingleMarkers(corners, MARKER_LENGTH, CAMERA_MATRIX, DIST_COEFFS)

        #Turn Rotation to world frame
        R_marker_to_cam, _ = cv2.Rodrigues(rvecs)
        R_cam_to_marker = R_marker_to_cam.T
        #Turn translation to world frame
        t_cam_to_marker = -R_cam_to_marker @ tvecs #The position of the camera in the marker's coordinate frame
        rvec_cam_to_marker, _ = cv2.Rodrigues(R_cam_to_marker) #The rotation of the camera in relation to the marker (in marker frame)
        x, y, z = rotationMatrixToEulerAngles(rvec_cam_to_marker) #Roll, pitch, yaw

        #Make ID msg(Remove if not needed)
        idmsg = String()
        idmsg.data = ' '.join(tagids.flatten().tolist())

        #Make position msg where x, y, z are the distance of the camera compared to the marker
        # rz is roll, ry is pitch, rz is yaw, published for debugging or other purposes
        msg = Int32()
        msg.x, msg.y, msg.z = t_cam_to_marker
        msg.rx, msg.ry, msg.rz = x, y, z

        # Publish
        self.tagid_pub.publish(idmsg)
        self.position_pub.publish(msg)
    
    """
    Function Description: Passing in a grayscale cv2 image and detects the AR tag based on the aruco 5x5 dictionary
    with numbers 1-100 (WARNING: IF TAG NOT FULLY VISIBLE AS SQUARE WILL NOT WORK)

    Input: Cv2 Image
    Output: numpy array of TAGIDs (may be multiple)
    """
    def detect_id(self, image):
        image = cv2.convertScaleAbs(gray, alpha=1.0, beta=-50)
        image = cv2.erode(image, kernel)


        aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_5X5_100)

        detectorParams = aruco.DetectorParameters()
        detectorParams.aprilTagCriticalRad = 0.1 #default 0.17
        detectorParams.aprilTagMaxLineFitMse = 100 #default 10
        detectorParams.aprilTagMaxNmaxima = 15 #default 10
        detectorParams.polygonalApproxAccuracyRate = 0.01 #default 0.03
        detectorParams.useAruco3Detection = True
        detector = aruco.ArucoDetector(aruco_dict, detectorParams)
        corners, ids, rejected = detector.detectMarkers(gray)

        return ids.flatten(), corners

def main(args=None):
    rclpy.init(args=args)
    detector = ARTagDetector()
    detector.get_logger.info('ARTag detector initialized')
    try:
        rclpy.spin(detector)
    except KeyboardInterrupt:
        print("ARTag detector shutting down")
    except Exception as e:
        print(f"ERROR: {e}")
    finally:
        detector.destroy_node()
        rclpy.shutdown()


if __name__=='__main__':#Unneeded if you only ever run the node directly with ros2 run 
    main()

